readme: 
* reference: 
  * [Using FastText models (not vectors) for robust embeddings](https://www.kaggle.com/mschumacher/using-fasttext-models-for-robust-embeddings)
  * [Medium - Build your own Text classification with less than 25 lines of code using fasttext](https://medium.com/@ravindraprasad/build-your-own-text-classification-in-less-than-25-lines-of-code-using-fasttext-dae7229f80f9)
  * [Medium - Learning FastText](https://towardsdatascience.com/fasttext-ea9009dba0e8)
  * [Text Classification & Word Representations using FastText (An NLP library by Facebook)](https://www.analyticsvidhya.com/blog/2017/07/word-representations-text-classification-using-fasttext-nlp-facebook/)

In [2]:
import pandas as pd
import numpy as np
import random
import os
os.chdir('/Users/liyuan/desktop/CSAir/codes')
import fastText 

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
from keras.layers import Flatten
from keras.layers import Embedding
import tensorflow as tf

from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from pandas_ml import ConfusionMatrix

from semi_supervise import Semi_Supervise

Using TensorFlow backend.


In [3]:
# ---- prepare data from user reviews to train FastText model ----
# load data
ss = Semi_Supervise()
labeled_data = ss.load_labeled_data('../res/labeled_data_with_without_tk.csv')
# load unlabeled data
# unlabeled_data = ss.load_unlabeled_data_csv('../res/unlabeled_review_5000.csv')

# load more unlabeled data ..
unlabeled_data = ss.load_unlabeled_data_csv('../res/unlabeled_review_反馈数据.csv')
# concatenate labeled and unlabeled data
data_concat = ss.concat_data()
data_concat.head()

there are 1550 examples in labeled dataset


/Users/liyuan/Desktop/CSAir/codes/semi_supervise.py:53: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  self.data_concat = pd.concat([self.labeled_data, self.unlabeled_data],ignore_index=True)


,review,review_tokens,label,label_encoded
0,NaN,机票 改签 在线 申请 邮寄 行程 操作,N/A,NaN
1,NaN,购买 机票 优惠 劵,N/A,NaN
2,NaN,网上 选 座位,N/A,NaN
3,NaN,开具 电子 发票 程序 自动 去掉 正确 应为 劳动保护 杂志社,N/A,NaN
4,我到现时用南方的app订票付款方式不能用到微信或支付宝，巳经多次找你们都解决不了问题？,现时 南方 订票 付款 方式 用到 微信 支付宝 巳 找 解决不了,预订,9.0


In [4]:
# output data to txt file => data would be feed into fasttext
train_data = data_concat.review_tokens
train_data.to_csv('../res/sampled_data_fasttext.txt', index = False)

# Finished: locally train fasttext using labeled + unlabeled data (5000)
# use command below to locally train skip-gram fasttext model: 
# ./fasttext skipgram -input ../fasttext_train_data/sampled_data_fasttext.txt -output ../fasttext_train_data/model

# train cbow version:
# ./fasttext cbow -input ../fasttext_train_data/sampled_data_fasttext.txt -output ../fasttext_train_data/model_cbow

In [5]:
class fasttext():
    def __init__(self):
        self.embeddings_index = {}
        self.MAX_SEQUENCE_LENGTH = 1000
        self.MAX_NUM_WORDS = 20000
        self.EMBEDDING_DIM = 300
        self.VALIDATION_SPLIT = 0.2
        self.labels_index = {}
        self.word_index  = {}
    
    def load_pretrained_model(self, model_path):
        model_pretrained = fastText.load_model(model_path) 
        return model_pretrained
    
    def prepare_data(self,data_file_path):
        self.all_labeled_data = pd.read_csv(data_file_path)
        self.texts = self.all_labeled_data.review_tokens.astype('str').values
        self.labels = self.all_labeled_data.label_encoded.values
        
        # get a dictionary that map each original label to its encoded label, e.g., {'中转': 0,...}
        for label in self.all_labeled_data.label.unique().tolist():
            self.labels_index[label] = self.all_labeled_data[self.all_labeled_data['label'] == label]['label_encoded'].unique()[0]

        tokenizer = Tokenizer(nb_words=self.MAX_NUM_WORDS)
        tokenizer.fit_on_texts(self.texts)
        sequences = tokenizer.texts_to_sequences(self.texts)

        self.word_index = tokenizer.word_index
        print('Found %s unique tokens.' % len(self.word_index))

        self.data = pad_sequences(sequences, maxlen=self.MAX_SEQUENCE_LENGTH)
        print('Shape of data tensor:', self.data.shape)
        print('Shape of label tensor:', self.labels.shape)
        
        # Converts a class vector (integers) to binary class matrix
        self.labels = to_categorical(np.asarray(self.labels))
        print('Shape of data tensor:', self.data.shape)
        print('Shape of label tensor:', self.labels.shape)

        # split the data into a training set and a validation set
        self.indices = np.arange(self.data.shape[0])
        np.random.shuffle(self.indices)
        self.data = self.data[self.indices]
        self.labels = self.labels[self.indices]
        nb_validation_samples = int(self.VALIDATION_SPLIT * self.data.shape[0])

        self.X_train = self.data[:-nb_validation_samples]
        self.y_train = self.labels[:-nb_validation_samples]
        self.X_val = self.data[-nb_validation_samples:]
        self.y_val = self.labels[-nb_validation_samples:]
        return  self.X_train, self.y_train, self.X_val, self.y_val
    
    def get_embedding_matrix(self):
        # 据得到的字典生成上文所定义的词向量矩阵
        embedding_matrix = np.zeros((len(self.word_index) + 1, self.EMBEDDING_DIM))
        for word, i in self.word_index.items():
            embedding_vector = self.embeddings_index.get(word)
            # updated:
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector
        return embedding_matrix
    
    def setup_neural_net(self):
        # get word embedding matrix
        self.embedding_matrix = self.get_embedding_matrix()

        # 将这个词向量矩阵加载到Embedding层
        embedding_layer = Embedding(len(self.word_index) + 1,
                                    self.EMBEDDING_DIM,
                                    weights=[self.embedding_matrix],
                                    input_length=self.MAX_SEQUENCE_LENGTH,
                                    trainable=False)
        
        # 使用一个小型的1D卷积解决分类问题
        sequence_input = Input(shape=(self.MAX_SEQUENCE_LENGTH,), dtype='int32')
        embedded_sequences = embedding_layer(sequence_input)
        x = Conv1D(128, 5, activation='relu')(embedded_sequences)
        x = MaxPooling1D(5)(x)
        x = Conv1D(128, 5, activation='relu')(x)
        x = MaxPooling1D(5)(x)
        x = Conv1D(128, 5, activation='relu')(x)
        x = MaxPooling1D(35)(x)  # global max pooling
        x = Flatten()(x)
        x = Dense(128, activation='relu')(x)
        preds = Dense(len(self.labels_index), activation='softmax')(x)
        return sequence_input,preds
    
    
    def train_data(self,X_train,y_train,X_val,y_val):
        sequence_input,preds = self.setup_neural_net()
        model = Model(sequence_input, preds)
        model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['acc'])
        # can change the number of epoch accordingly
        # 7 generates the best performance
        model.fit(X_train, y_train, validation_data=(X_val, y_val),
                  nb_epoch=7, batch_size=128)  
        
        # evaluate model using model.evaluate()
        scores = model.evaluate(X_val, y_val, verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        
        # get predicted class label
        self.output = model.predict(X_val)
        predicted_label_list = self.get_pred_label(self.output)
        return predicted_label_list
    
    
    def get_pred_label(self,output):
        '''get predicted class label based on prediction output'''
        predicted_label_list = []
        for i in range(len(output)):
            predicted_label = output[i].argmax(axis=-1)
            predicted_label_list.append(predicted_label)        
        return predicted_label_list
    
    
    def incorporate_pred_label(self):
        '''return prediction results back to df'''
        # indices is a numpy array, need to convert to a list of indices before feed into df to get sub df 
        # recreate df based on the shuffled indices
        indices = self.indices
        all_labeled_data = self.all_labeled_data.iloc[list(self.indices)]
        nb_validation_samples = int(self.VALIDATION_SPLIT * self.data.shape[0])
        print(nb_validation_samples)
        # need to get the indices of the validation data
        train_val_bound = self.data.shape[0] - nb_validation_samples
        # get validation dataset
        val_df = all_labeled_data[train_val_bound:]
        return val_df

    def map_label(self,df,predicted_label_list):
        '''map predicted labels to original class'''
        # print(predicted_label_list[:10])
        label_dct = self.labels_index
        df['pred_label_encodes'] = predicted_label_list
        # get reversed labels_index dictionary
        reversed_label_dct = {}
        for i in range(len(label_dct)):
            reversed_label_dct[list(label_dct.values())[i]] = list(label_dct.keys())[i]

        # map predicted labels
        pred_label = [reversed_label_dct.get(label) for label in predicted_label_list]
        df['pred_label'] = pred_label
        return df
    
    
    def evaluate_performance(self,val_df):
        # evaluate performance
        y_val_true = val_df.label.values
        y_val_pred = val_df.pred_label.values
        self.get_confusion_matrix(y_val_true,y_val_pred) 
        
    
    def get_confusion_matrix(self,y_test,y_pred):
        '''get tp,tn,fp,fn for each class'''
        cm = ConfusionMatrix(y_test, y_pred)
        cm.print_stats()
        
        
    def over_sampling(self):
        '''modeling after over sampling'''
        smote = SMOTE('minority')
        X_train_sm, y_train_sm = smote.fit_sample(self.X_train,self.y_train)
        print(X_train_sm.shape, y_train_sm.shape)
        
        # fit model based on new data set
        predicted_label_list = self.train_data(X_train_sm,y_train_sm,X_val,y_val)
        return predicted_label_list
    

In [6]:
ft = fasttext()
# model = ft.load_pretrained_model('fasttext_train_data/model.bin')
# model = ft.load_pretrained_model('fasttext_train_data/model_cbow.bin')

# load pretrained from official site ..
model = ft.load_pretrained_model('../Source_Data/cc.zh.300.bin')

In [7]:
# get more model methods
print('dimention of word vector:',model.get_dimension())
# load word vector
model.get_word_vector('航班').astype('float32')

dimention of word vector: 300


array([-9.74254776e-03,  2.12122593e-02,  4.88131315e-01, -6.32701069e-02,
       -7.33671244e-03, -2.18946919e-01, -1.77119195e-01,  2.03831811e-02,
       -2.26945549e-01,  1.36024758e-01, -1.81659728e-01,  1.12053066e-01,
        1.12769455e-01,  8.50891769e-02,  2.61089087e-01,  1.55254016e-02,
       -8.70296210e-02,  1.53820127e-01,  1.77355576e-02,  1.37629956e-01,
       -6.68650493e-02, -3.91467549e-02,  7.03171268e-02, -1.34001851e-01,
       -5.05742505e-02, -1.98219940e-01,  5.21637499e-02,  2.09236205e-01,
        3.95330228e-02,  9.62841138e-02,  7.60198310e-02, -5.18575832e-02,
        1.30827308e-01, -1.79477498e-01,  9.56601351e-02, -2.05786884e-01,
        2.56274760e-01, -1.52252838e-01,  1.57351568e-01, -2.99355295e-02,
        1.20841317e-01, -6.54572770e-02, -2.32028976e-01,  1.31989181e-01,
        2.59281192e-02,  3.18626165e-01, -8.46268050e-03,  1.44843146e-01,
        9.82890576e-02, -7.24061728e-02,  1.34834737e-01,  2.83483323e-02,
        2.68899530e-01,  

In [9]:
X_train, y_train, X_val, y_val = ft.prepare_data('../res/labeled_data_with_without_tk.csv')

/anaconda3/envs/nlp/lib/python3.6/site-packages/keras_preprocessing/text.py:177: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 4747 unique tokens.
Shape of data tensor: (1551, 1000)
Shape of label tensor: (1551,)
Shape of data tensor: (1551, 1000)
Shape of label tensor: (1551, 10)


In [10]:
predicted_label_list = ft.train_data(X_train,y_train,X_val,y_val)

/anaconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:99: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 1241 samples, validate on 310 samples
Epoch 1/7
1241/1241 [==============================] - 37s 30ms/step - loss: 2.2986 - acc: 0.1813 - val_loss: 2.2958 - val_acc: 0.2258
Epoch 2/7
1241/1241 [==============================] - 28s 23ms/step - loss: 2.2927 - acc: 0.1990 - val_loss: 2.2916 - val_acc: 0.1581
Epoch 3/7
1241/1241 [==============================] - 28s 23ms/step - loss: 2.2881 - acc: 0.1934 - val_loss: 2.2877 - val_acc: 0.1581
Epoch 4/7
1241/1241 [==============================] - 27s 22ms/step - loss: 2.2838 - acc: 0.1934 - val_loss: 2.2841 - val_acc: 0.1581
Epoch 5/7
1241/1241 [==============================] - 29s 23ms/step - loss: 2.2796 - acc: 0.1934 - val_loss: 2.2806 - val_acc: 0.1581
Epoch 6/7
1241/1241 [==============================] - 28s 22ms/step - loss: 2.2755 - acc: 0.1829 - val_loss: 2.2771 - val_acc: 0.1581
Epoch 7/7
1241/1241 [==============================] - 29s 24ms/step - loss: 2.2715 - acc: 0.1942 - val_loss: 2.2738 - val_acc: 0.1581
acc: 15.

In [11]:
# check label index
print('label encoding dictionary:', ft.labels_index)

val_df = ft.incorporate_pred_label()
val_df = ft.map_label(val_df,predicted_label_list)
val_df.head()

# evaluate performance
ft.evaluate_performance(val_df)

label encoding dictionary: {'计划': 7, '机上': 5, '中转': 0, '售后': 3, '预订': 9, '设计': 8, '出发': 1, '性能': 4, '行程': 6, '到达': 2}
310


/anaconda3/envs/nlp/lib/python3.6/site-packages/pandas_ml/confusion_matrix/abstract.py:66: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  df = df.loc[idx, idx.copy()].fillna(0)  # if some columns or rows are missing
/anaconda3/envs/nlp/lib/python3.6/site-packages/pandas_ml/confusion_matrix/stats.py:60: FutureWarning: supplying multiple axes to axis is deprecated and will be removed in a future version.
  num = df[df > 1].dropna(axis=[0, 1], thresh=1).applymap(lambda n: choose(n, 2)).sum().sum() - np.float64(nis2 * njs2) / n2
/anaconda3/envs/nlp/lib/python3.6/site-packages/pandas_ml/confusion_matrix/bcm.py:236: RuntimeWarning: invalid value encountered in double_scalars
  return(np.float64(self.TP) / self.PositiveTest)
/anaconda3/envs/nlp/lib/python3.6/site-packages/p

Confusion Matrix:

Predicted  中转  出发  到达  售后  性能   机上  行程  计划  设计  预订  __all__
Actual                                                     
中转          0   0   0   0   0   32   0   0   0   0       32
出发          0   0   0   0   0   70   0   0   0   0       70
到达          0   0   0   0   0   29   0   0   0   0       29
售后          0   0   0   0   0   25   0   0   0   0       25
性能          0   0   0   0   0   26   0   0   0   0       26
机上          0   0   0   0   0   49   0   0   0   0       49
行程          0   0   0   0   0   12   0   0   0   0       12
计划          0   0   0   0   0    8   0   0   0   0        8
设计          0   0   0   0   0   17   0   0   0   0       17
预订          0   0   0   0   0   42   0   0   0   0       42
__all__     0   0   0   0   0  310   0   0   0   0      310


Overall Statistics:

Accuracy: 0.15806451612903225
95% CI: (0.11928240540905369, 0.2035282629919941)
No Information Rate: ToDo
P-Value [Acc > NIR]: 1.0
Kappa: 0.0
Mcnemar's Test P-Value: ToDo


Class

/anaconda3/envs/nlp/lib/python3.6/site-packages/pandas_ml/confusion_matrix/bcm.py:259: RuntimeWarning: invalid value encountered in double_scalars
  return(np.float64(self.TN) / self.NegativeTest)
/anaconda3/envs/nlp/lib/python3.6/site-packages/pandas_ml/confusion_matrix/bcm.py:337: RuntimeWarning: invalid value encountered in double_scalars
  return(np.float64(self.FNR) / self.TNR)
/anaconda3/envs/nlp/lib/python3.6/site-packages/pandas_ml/confusion_matrix/bcm.py:251: RuntimeWarning: invalid value encountered in double_scalars
  return(np.float64(self.FN) / self.NegativeTest)


summary: using labeled and unlabeled data (~25000 reviews) => poorly predict, by only predicting 出发 （the majority class label)